In [2]:
import pathlib
import pandas as pd

# read in 4016852c-d13f-4292-9fa8-6b86d7f0b9b8 from the dataset folder into a pandas dataframe
affiliate_leads2 = pd.read_csv(
    str(pathlib.Path.cwd() / "4016852c-d13f-4292-9fa8-6b86d7f0b9b8"))

C:\Users\Colin.Hanley\AppData\Local\Temp\ipykernel_14384\2987357974.py:5: DtypeWarning: Columns (40,53,66,98,100,108,134,140,158,161,162,168,169,170,201,203,204,210,212,214,215,247,253,256,262,299,349,358,400,454,467,469,473,511,535,545,549,587,594,595,645,651,655,673,677,687,699,700,704,755,770,792,795,797,819,870,893,898,899,900,904,909,914,921,922,926,933,938,944,946,947,950,954,958,968,976,985,990,992,998,1009,1019,1028,1042,1068,1069,1095,1100,1113,1115,1131,1132,1137,1142,1155,1157,1166,1171,1173,1183,1184,1186,1187,1190,1194,1207,1216,1221,1223,1252,1258,1260,1261,1268,1275,1277,1279,1281,1283,1286,1290,1291,1292,1293,1294,1297,1326,1330,1334,1340,1341,1342,1343,1344,1348,1353,1383,1384,1392,1409,1412,1414,1417,1423,1426,1428,1433,1435,1443,1447,1448,1455,1460,1461,1466,1470,1476,1490,1491,1492,1494,1501,1504,1509,1517,1518,1519,1523,1533,1539,1546,1573,1582,1591,1607,1611,1615,1623,1631,1635,1648,1650,1661,1664,1686,1696,1702,1704,1705,1708,1711,1714,1740,1741,1748,1762,1764,17

In [3]:
from datascience_core.data_transformation import ColumnFiller, ColumnRenamer, ValueReplacer
import os
import pickle


class CatTypeCoverter:
    pass


class ADModel:
    def __init__(self, fmt="speedy"):

        self.fmt = fmt
        self.model = pickle.load(str(pathlib.Path.cwd() / "AD3_v1.pkl")
        )
        # cat feature names
        self.make_categorical = list(
            pd.DataFrame(data=self.model.get_cat_feature_indices(), columns=["indices"])
            .merge(
                pd.DataFrame(
                    data=self.model.feature_names_, columns=["features"]
                ).reset_index(),
                how="left",
                left_on="indices",
                right_on="index",
            )["features"]
            .values
        )

        # get top 100 features
        dfTop100Features = pd.DataFrame.from_dict(
            data={
                "features": self.model.feature_names_,
                "values": self.model.feature_importances_,
            }
        ).sort_values(by="values", ascending=False)[0:100]

        self.lsTop100Features = dfTop100Features["features"].values

        # preprocessing steps
        pre_step = ColumnRenamer(fmt=fmt)
        step1 = ColumnFiller(
            ["FE Lender"], fillValue="247 Money", enforce=False, fmt=fmt
        )
        step2 = ColumnFiller(
            self.model.feature_names_,
            self.lsTop100Features,
            fmt=fmt,
        )
        step3 = ValueReplacer(
            lsUnwanted=[
                "M",
                "C",
                "{ND}",
                "ND",
                "OB",
                "Not Found",
                "{OB}",
                "T",
                "__",
                -999997,
                -999999,
                999999,
                999997,
                -999997.0,
                -999999.0,
                999999.0,
                999997.0,
                "-999997",
                "-999999",
                "999999",
                "999997",
                "-999997.0",
                "-999999.0",
                "999999.0",
                "999997.0",
            ]
        )
        step4 = CatTypeCoverter(self.make_categorical, fmt=fmt)

        self.steps = [pre_step, step1, step2, step3, step4]

    def preprocess(self, dfIn):
        dfOut = copy.deepcopy(dfIn)
        for step in self.steps:
            dfOut = step.run_step(dfOut)
        return dfOut

    def make_prediction(self, dfIn):
        if self.fmt == "speedy":
            return [
                self.model.predict_proba(
                    dfIn.loc[self.model.feature_names_, "0"].values
                )[1]
            ]
        else:
            return list(self.model.predict_proba(dfIn[self.model.feature_names_])[:, 1])

C:\Users\Colin.Hanley\AppData\Local\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [ ]:
import gc
import numpy as np

def predict_dataset_in_chunks(data, chunk_size=10000):
    total_rows = len(data)
    num_chunks = int(np.ceil(total_rows / chunk_size))

    model = ADModel()
    predictions_list = []

    for i in range(num_chunks):
        start_index = i * chunk_size
        end_index = min((i + 1) * chunk_size, total_rows)
        chunk_data = data[start_index:end_index]
        print(f"Processing chunk {i+1}/{num_chunks}...")

        preprocessed_data = model.preprocess(chunk_data)
        predictions = model.make_prediction(preprocessed_data)
        predictions_list.append(predictions)

        del preprocessed_data, predictions
        gc.collect()

    return np.array(predictions_list).flatten()

In [ ]:
predictions = predict_dataset_in_chunks(affiliate_leads2,10000)

In [4]:
affiliate_leads2

,ApplicationId,App.MainCustomerId,App.ApplicationDate,App.AmountToFinance,App.VehicleType,App.Title,App.Gender,App.AgeAtApplication,App.Maritalstatus,App.DrivingLicenceType,...,DateFirstRegistered,Make,GlassAdjustedPrice,Mileage,FuelType,Transmission,Colour,IsWrittenAgreement,IsDocOut,IsApproval
0,12857471,12492273,2022-12-01 08:55:00,7995.0,Car,Mr,Male,57.0,Married,International Licence,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,True
1,12858185,12492987,2022-12-01 12:13:00,12995.0,Car,Mr,Male,19.0,Single,Full UK Driving Licence,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False
2,12858412,12493214,2022-12-01 13:16:00,5700.0,Car,Mr,Male,54.0,Married,Full UK Driving Licence,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False
3,12858430,12493232,2022-12-01 13:21:00,17500.0,Car,Mr,Male,56.0,Married,Full UK Driving Licence,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,True
4,12858682,12493484,2022-12-01 14:27:00,6500.0,Van,Mr,Male,41.0,Married,Full UK Driving Licence,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
580101,13541028,13162527,2023-04-24 13:04:00,7900.0,Car,Mrs,Female,32.0,Cohabiting,Full UK Driving Licence,...,2017-03-29 00:00:00,PEUGEOT,0,71182.0,PETROL,MANUAL 5 GEARS,WHITE,True,True,True
580102,13541028,13162527,2023-04-24 13:04:00,7900.0,Car,Mrs,Female,32.0,Cohabiting,Full UK Driving Licence,...,2017-03-29 00:00:00,PEUGEOT,0,71182.0,PETROL,MANUAL 5 GEARS,WHITE,True,True,True
580103,13138667,12775547,2023-02-02 21:32:00,30000.0,Car,Mr,Male,50.0,Cohabiting,Full UK Driving Licence,...,2016-07-30 00:00:00,MERCEDES-BENZ,0,76000.0,PETROL,AUTO 7 GEARS,GREY,True,True,True
580104,12922661,12557904,2022-12-18 10:40:00,5000.0,Car,Mr,Male,36.0,Married,Full UK Driving Licence,...,2014-07-03 00:00:00,NISSAN,0,98419.0,DIESEL,MANUAL 5 GEARS,BLACK,True,True,True


In [8]:
affiliate_leads2["UTMSource"][affiliate_leads2["UTMMedium"] == "affiliate"].unique()

array(['ckdirect', 'experian', 'youngcardriver', nan, 'parkers',
       'totallymoney', 'lendingexpert', 'awin', 'affiliatewindow'],
      dtype=object)